In [2]:
import keras
import numpy as np

from keras.models import load_model
from keras.utils import to_categorical


# load trained model for visualization
cae = load_model('trained_cae.h5')

encoder = load_model('trained_encoder.h5')
decoder = load_model('trained_decoder.h5')


model = decoder


M = 128
L = 5
numepochs = 25


# One hot encoding
rawdata = np.zeros(M)
i = 0
while(i<M):
    rawdata[i] = i
    i += 1

data = to_categorical(rawdata)
#from numpy import reshape
data = data.astype(int)
ydata = np.reshape(data, (data.shape[0], 1, 1, data.shape[1]))
yvaldata = ydata
xdata = np.reshape(data, (data.shape[0], 1, 1, data.shape[1]))
xvaldata = xdata

print('done')

Using TensorFlow backend.


done


In [3]:
class_idx = ydata[0]
#indices = y_test[:, 0]
#indices = np.where(ydata[:, class_idx] == 1.)[0]
indices = ydata

idx = ydata[0]
#idx = indices[0]
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (128, 6)
#plt.imshow(idx[:])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 5, 5, 1)           0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 20, 20, 1)         0         
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 20, 20, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 256)       6656      
_________________________________________________________________
batch_normalization_1 (Batch (None, 20, 20, 256)       1024      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 256)       590080    
__________

In [4]:
activations = encoder.predict(xdata)
#print(activations[0])
#print(activations[0])
inp = activations[0]

In [5]:
dim1 = np.sqrt(M)
if (np.mod(dim1, 1) == 0):
    dim2 = dim1
else:
    dim1 = np.sqrt(M/2)
    dim2 = M/dim1



In [11]:
from vis.visualization import visualize_saliency
from vis.utils import utils
from keras import activations

layer_idx = utils.find_layer_idx(model, 'conv2d_2')
print(layer_idx)
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

6


In [12]:
for index in np.arange(np.min(32,M)):    
    activations = encoder.predict(xdata)
    #print(activations[0])
    #print(activations[index])
    inp = activations[index]

    f, ax = plt.subplots(1, 4)
    # displays input
    ax[0].set_title(index)
    ax[0].imshow(inp[:, ..., 0], cmap='binary')
    
    for i, modifier in enumerate([None, 'guided', 'relu']):
        grads = visualize_saliency(model, layer_idx, filter_indices=None, 
                                   seed_input=inp, backprop_modifier=modifier)
        if modifier is None:
            modifier = 'vanilla'
        ax[i+1].set_title(modifier)    
        ax[i+1].imshow(grads, cmap='binary')

AxisError: axis 128 is out of bounds for array of dimension 0